데이터 불러오기

In [ ]:
import pandas as pd

products = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CP1/products.csv')
orders = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CP1/orders.csv')
order_products_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CP1/order_products__train.csv')
departments = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CP1/departments.csv')
aisles = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CP1/aisles.csv')
order_products_prior = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CP1/order_products__prior.csv')

print('products shape :', products.shape)
print('orders shape :', orders.shape)
print('order_products_train shape :', order_products_train.shape)
print('departments shape :', departments.shape)
print('aisles shape :', aisles.shape)
print('order_products_prior shape :', order_products_prior.shape)

products shape : (49688, 4)
orders shape : (3421083, 7)
order_products_train shape : (1384617, 4)
departments shape : (21, 2)
aisles shape : (134, 2)
order_products_prior shape : (32434489, 4)


랜덤하게 데이터를 자를 경우, 유저가 가지는 주문 특성이 왜곡이 될 것 같아서 유저를 기준으로 데이터를 축소해보았습니다. 밑의 코드와 같이 user의 정보가 담겨있는 orders테이블에서 1~1000번에 해당되는 유저의 주문 정보까지 데이터를 축소시켜보았습니다. 그리고 eval_set 특성에 맞게 prior, train, test로 분리하여 prior 데이터로 추천시스템을 구현해보고자 합니다.

In [ ]:
orders_prior = orders[(orders['user_id']<=1000) & (orders['eval_set']=='prior')]
orders_train = orders[(orders['user_id']<=1000) & (orders['eval_set']=='train')]
orders_test = orders[(orders['user_id']<=1000) & (orders['eval_set']=='test')]

필요한 정보는 user_id, order_id뿐이므로 나머지 특성들을 드랍했습니다.

In [ ]:
orders_prior.drop(['eval_set', 'order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order'], axis=1, inplace=True)
orders_train.drop(['eval_set', 'order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order'], axis=1, inplace=True)
orders_test.drop(['eval_set', 'order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order'], axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
orders_prior

order_id  user_id
0       2539329        1
1       2398795        1
2        473747        1
3       2254736        1
4        431534        1
...         ...      ...
16139   2700013     1000
16140   2003258     1000
16141    568709     1000
16142     18216     1000
16143   2885089     1000

[15145 rows x 2 columns]

In [ ]:
orders_train

order_id  user_id
10      1187899        1
25      1492625        2
49      2196797        5
74       525192        7
78       880375        8
...         ...      ...
15964    910179      991
15986   3289231      992
16011    637980      994
16052   1982039      996
16144    903956     1000

[643 rows x 2 columns]

In [ ]:
orders_test

order_id  user_id
38      2774568        3
44       329954        4
53      1528013        6
96      1376945       11
102     1356845       12
...         ...      ...
16000   2279794      993
16047   1248981      995
16070   1516872      997
16097    533523      998
16136     20345      999

[357 rows x 2 columns]

혹시모를 램터짐을 방지하기 위해서 기존의 orders 테이블을 제거해주었습니다.

In [ ]:
del orders

유저수가 제대로 1000이 나오는지 확인해보았습니다.

In [ ]:
len(orders_prior['user_id'].value_counts())

1000

# 단순 장바구니 코사인 유사도
총 2가지 방식으로 코사인 유사도를 통해 장바구니 추천을 구현해보았습니다.
1. 상품명 벡터

  유저가 주문별로 장바구니에 담은 상품들을 리스트로 만들고 해당 리스트를 문장으로 만들어 BOW기반의 CounterVecterizer를 통해 벡터화시킨 후 코사인 유사도를 통해 구현하는 방식입니다.
  
  단, 상품의 정보가 포함되어 있지 않아 의미있는(?) 유사도를 구할 수가 없습니다. 예로 들어서 super_monkey, power_monkey라는 상품명의 경우, 둘이 비슷한 상품임을 예측할 수 있지만 상품명이 다르기에 벡터는 완전히 달라지고 낮은 유사도를 결과로 예측할 것입니다.
2. 상품의 메타데이터 벡터

  유저가 주문별로 장바구니에 담은 상품들을 리스트로 만드는 것까지는 같지만 해당 리스트를 바로 문장으로 만드는 것이 아닌 각 상품들의 정보가 포함될 수 있도록 상품을 대표하는 소분류(aisle), 대분류(department)를 가져와서 문장으로 만드는 것입니다. 예를 들면 '초코파이'인 경우 소분류로 '과자', 대분류로 '음식'이기 때문에 '초코파이'를 문장에 넣는 것이 아닌 '과자 음식'을 문장에 넣는 것입니다.

  상품의 정보가 포함되어 있기 때문에 의미있는 유사도를 구할 수 있습니다. 예를 들면 '킹왕짱 케이크', '맛있는 케이크'는 각각 cookies/cake, pan/cake에 속할 경우, cookies와 pan이 다르지만 cake가 같기 때문에 비슷한 벡터로 벡터화 될 수 밖에 없고 이는 높은 유사도로 결과가 나오게 될 것입니다.


In [ ]:
order_products_prior.drop(['add_to_cart_order', 'reordered'], axis=1, inplace=True)
orders_prior = orders_prior.merge(order_products_prior, on='order_id')

1000명의 유저데이터로 주문번호, 유저번호, 상품번호를 한 테이블로 모으면

In [ ]:
orders_prior

order_id  user_id  product_id
0        2539329        1         196
1        2539329        1       14084
2        2539329        1       12427
3        2539329        1       26088
4        2539329        1       26405
...          ...      ...         ...
150145   2885089     1000         663
150146   2885089     1000       28849
150147   2885089     1000       44632
150148   2885089     1000       19660
150149   2885089     1000       38374

[150150 rows x 3 columns]

### 1. 상품명 벡터

유저별로 주문한 번호에 따라 장바구니에 어떤 상품들이 들어있는지를 테이블화 합니다.

In [ ]:
# prior = order_products_prior.merge(products, on='product_id')
prior = orders_prior.groupby(['user_id', 'order_id'])['product_id'].apply(list)

In [ ]:
prior = prior.reset_index()
prior

user_id  order_id                                         product_id
0            1    431534  [196, 12427, 10258, 25133, 10326, 17122, 41787...
1            1    473747                  [196, 12427, 10258, 25133, 30450]
2            1    550135                  [196, 10258, 12427, 25133, 13032]
3            1   2254736                  [196, 12427, 10258, 25133, 26405]
4            1   2295261           [49235, 46149, 25133, 196, 10258, 12427]
...        ...       ...                                                ...
15140     1000   1456368  [26165, 28465, 14870, 30492, 36865, 37646, 496...
15141     1000   2003258  [26165, 14870, 36865, 47734, 30492, 37646, 273...
15142     1000   2700013  [26165, 36865, 28465, 30492, 49683, 47734, 196...
15143     1000   2885089  [26165, 30492, 14870, 28465, 49683, 39877, 687...
15144     1000   3045856  [30492, 14870, 36865, 28465, 26165, 19660, 188...

[15145 rows x 3 columns]

In [ ]:
def tospc(arg):
  return arg.replace(' ', '')

def tostr(arg):
  return ' '.join([products.loc[products[products['product_id']==i].index, 'no_spc_product_name'].values[0] for i in arg])

products테이블에서 product_name은 공백을 포함하여 표현하고 있는데 벡터화를 위해 모든 공백을 없앤 no_spc_product_name칼럼을 따로 만들어줍니다. 그 이유는 CounterVerterizer화를 할 건데 상품명이 공백을 기준으로 짤리게 된다면 전혀 다른 상품이지만 유사한 상품이라고 오해할 수 있기 때문에 이를 방지하기 위함입니다. 예를 들면 Chocolate Sandwich Cookies와 Mintchocolate Sand Cookies있지만 Mintchocolate Sand Cookies의 경우 만약 샌드위치 쿠기가 아닌 진짜 샌드 쿠키, 즉 모래 쿠키로서 모래가 되어버린다면 둘은 완전히 다른 상품이지만 초코, 쿠기만 보고서 높은 유사도가 나오게 되는 끔찍한 일이 벌어질 것입니다. 그래서 제품명 전체를 하나의 벡터요소로 만들어주기 위해 공백을 모두 제거해줍니다.

In [ ]:
products['no_spc_product_name'] = products['product_name'].apply(tospc)
products.head()

product_id                                       product_name  aisle_id  \
0           1                         Chocolate Sandwich Cookies        61   
1           2                                   All-Seasons Salt       104   
2           3               Robust Golden Unsweetened Oolong Tea        94   
3           4  Smart Ones Classic Favorites Mini Rigatoni Wit...        38   
4           5                          Green Chile Anytime Sauce         5   

   department_id                                no_spc_product_name  
0             19                           ChocolateSandwichCookies  
1             13                                    All-SeasonsSalt  
2              7                   RobustGoldenUnsweetenedOolongTea  
3              1  SmartOnesClassicFavoritesMiniRigatoniWithVodka...  
4             13                             GreenChileAnytimeSauce

장바구니 리스트에 맞춰서 각 제품명들을 하나로 합쳐 문장으로 만들어줍니다.

In [ ]:
prior['product_list'] = prior['product_id'].apply(tostr)
prior.head()

user_id  order_id                                         product_id  \
0        1    431534  [196, 12427, 10258, 25133, 10326, 17122, 41787...   
1        1    473747                  [196, 12427, 10258, 25133, 30450]   
2        1    550135                  [196, 10258, 12427, 25133, 13032]   
3        1   2254736                  [196, 12427, 10258, 25133, 26405]   
4        1   2295261           [49235, 46149, 25133, 196, 10258, 12427]   

                                        product_list  
0  Soda OriginalBeefJerky Pistachios OrganicStrin...  
1  Soda OriginalBeefJerky Pistachios OrganicStrin...  
2  Soda Pistachios OriginalBeefJerky OrganicStrin...  
3  Soda OriginalBeefJerky Pistachios OrganicStrin...  
4  OrganicHalf&Half ZeroCalorieCola OrganicString...

CounterVertorizer를 통해 단순 빈도수 기반으로 벡터화를 시켜줍니다.(관사 등 자주 쓰이고 의미없는 단어없이 모두 고유명사로 표현되어있기에 TfIdfVertorizer는 의미가 없습니다.) 그리고 코사인 유사도 매트릭스를 구해줍니다.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

count_vec = CountVectorizer()
count_vec_matrix = count_vec.fit_transform(prior['product_list'])
cosine_sim_matrix = cosine_similarity(count_vec_matrix, count_vec_matrix)

최종적으로 구해진 코사인 유사도 매트릭스에 맞춰 특정 상품명을 넣으면 해당 상품과 유사한 상위 10개의 상품들을 리스트로 받을 수 있는 함수를 만들어줍니다.

In [ ]:
mapping = pd.Series(prior.index, index=prior['product_list'])

In [ ]:
def recommend_product_list(product_input):
    
    product_index = mapping[product_input]
    
    similarity_score = list(enumerate(cosine_sim_matrix[product_index]))
    similarity_score = sorted(similarity_score, key=lambda x: x[1],reverse=True)
    similarity_score = similarity_score[1:10]
    product_indices = [i[0] for i in similarity_score]
    
    return (prior['product_list'].iloc[product_indices])

input으로 들어가는 parameter의 형태는 장바구니 형태인 리스트이여야 하므로 특정 유저, 그 특정 유저가 주문한 주문 번호를 안다면 해당 장바구니 리스트를 반환받을 수 있는 함수도 정의해주어야합니다.

In [ ]:
def find_list(user_num, order_num):
  return prior[(prior['user_id']==user_num) & (prior['order_id']==order_num)]['product_list'].values[0]

user_id = 1, order_id = 431534인 장바구니와 유사한 장바구니 상위 10개

In [ ]:
recommend_product_list(find_list(1, 431534))

9      Soda OriginalBeefJerky Pistachios OrganicStrin...
1      Soda OriginalBeefJerky Pistachios OrganicStrin...
2      Soda Pistachios OriginalBeefJerky OrganicStrin...
3      Soda OriginalBeefJerky Pistachios OrganicStrin...
5      Soda Pistachios OriginalBeefJerky BagofOrganic...
4      OrganicHalf&Half ZeroCalorieCola OrganicString...
8      OriginalBeefJerky Soda Pistachios OrganicStrin...
977                             BagofOrganicBananas Soda
7      Soda ZeroCalorieCola MilkChocolateAlmonds 0%Gr...
Name: product_list, dtype: object

### 2. 상품의 메타데이터 벡터

상품별로 해당 상품을 설명할 수 있는 메타정보를 만들어주어야하므로 소분류, 대분류의 정보를 요약한 'meta'칼럼을 만들어 줍니다.

In [ ]:
df = products.merge(aisles, on='aisle_id')
df = df.merge(departments, on='department_id')
df['meta'] = df['aisle']+' '+df['department']
df.head()

product_id                                    product_name  aisle_id  \
0           1                      Chocolate Sandwich Cookies        61   
1          78               Nutter Butter Cookie Bites Go-Pak        61   
2         102                           Danish Butter Cookies        61   
3         172  Gluten Free All Natural Chocolate Chip Cookies        61   
4         285                    Mini Nilla Wafers Munch Pack        61   

   department_id                       no_spc_product_name          aisle  \
0             19                  ChocolateSandwichCookies  cookies cakes   
1             19             NutterButterCookieBitesGo-Pak  cookies cakes   
2             19                       DanishButterCookies  cookies cakes   
3             19  GlutenFreeAllNaturalChocolateChipCookies  cookies cakes   
4             19                  MiniNillaWafersMunchPack  cookies cakes   

  department                  meta  
0     snacks  cookies cakes snacks  
1     snacks  cookies cakes snacks  
2     snacks  cookies cakes snacks  
3     snacks  cookies cakes snacks  
4     snacks  cookies cakes snacks

똑같이 문장으로 만들어줍니다. 하지만 'meta'칼럼에서는 공백을 제거하지 않습니다. 왜냐하면 메타데이터는 정보를 담고 있기에 단어는 유사하지만 상품은 완전히 다른 케이스가 나올 수가 없기 때문입니다. 예를 들면 cookies cake와 sand cake는 말그대로 cookies, cake의 정보와 sand, cake의 정보를 둘 다 들고 있는 것이기에 쿠기로 만들었건 모래로 만들었건 무조건 cake일 수 밖에 없습니다. 

In [ ]:
def tometa(arg):
  return ' '.join([df[df['product_id']==i]['meta'].values[0] for i in arg])

In [ ]:
prior['meta_list'] = prior['product_id'].apply(tometa)
prior.head()

user_id  order_id                                         product_id  \
0        1    431534  [196, 12427, 10258, 25133, 10326, 17122, 41787...   
1        1    473747                  [196, 12427, 10258, 25133, 30450]   
2        1    550135                  [196, 10258, 12427, 25133, 13032]   
3        1   2254736                  [196, 12427, 10258, 25133, 26405]   
4        1   2295261           [49235, 46149, 25133, 196, 10258, 12427]   

                                        product_list  \
0  Soda OriginalBeefJerky Pistachios OrganicStrin...   
1  Soda OriginalBeefJerky Pistachios OrganicStrin...   
2  Soda Pistachios OriginalBeefJerky OrganicStrin...   
3  Soda OriginalBeefJerky Pistachios OrganicStrin...   
4  OrganicHalf&Half ZeroCalorieCola OrganicString...   

                                           meta_list  
0  soft drinks beverages popcorn jerky snacks nut...  
1  soft drinks beverages popcorn jerky snacks nut...  
2  soft drinks beverages nuts seeds dried fruit s...  
3  soft drinks beverages popcorn jerky snacks nut...  
4  cream dairy eggs soft drinks beverages package...

In [ ]:
count_vec = CountVectorizer()
count_vec_matrix = count_vec.fit_transform(prior['meta_list'])
cosine_sim_matrix = cosine_similarity(count_vec_matrix, count_vec_matrix)
mapping = pd.Series(prior.index, index=prior['meta_list'])

In [ ]:
def recommend_meta_list(product_input):
    
    product_index = mapping[product_input]
    
    similarity_score = list(enumerate(cosine_sim_matrix[product_index]))
    similarity_score = sorted(similarity_score, key=lambda x: x[1],reverse=True)
    similarity_score = similarity_score[1:10]
    product_indices = [i[0] for i in similarity_score]
    
    return prior['product_list'].iloc[product_indices]

In [ ]:
def find_meta(user_num, order_num):
  return prior[(prior['user_id']==user_num) & (prior['order_id']==order_num)]['meta_list'].values[0]

user_id = 1, order_id = 431534인 장바구니와 유사한 장바구니 상위 10개(단, 메타데이터로 유사도를 측정한...)

In [ ]:
recommend_meta_list(find_meta(1, 431534))

4119     Strawberries SeaSaltMacadamias Banana LowfatMi...
2704     ExtraFancyUnsaltedMixedNuts OrganicBlackSeedle...
1608     BagofOrganicBananas OrganicLargeExtraFancyFuji...
1609     BagofOrganicBananas OrganicGrannySmithApple Or...
2725     ExtraFancyUnsaltedMixedNuts RedSeedlessGrapes ...
2707     RedSeedlessGrapes ExtraFancyUnsaltedMixedNuts ...
9595     OrganicLargeExtraFancyFujiApple OrganicMedjool...
12276    FujiApples BagofOrganicBananas ExtraFancyUnsal...
6821     UnsweetenedAlmondmilk OrganicBabyCarrots Origi...
Name: product_list, dtype: object

# 단순 상품별 코사인 유사도

In [ ]:
# count_vec = CountVectorizer()
# count_vec_matrix = count_vec.fit_transform(df['meta'])
# cosine_sim_matrix = cosine_similarity(count_vec_matrix, count_vec_matrix)
# mapping = pd.Series(df.index, index=df['product_name'])

In [ ]:
# def recommend_products(product_input):
    
#     product_index = mapping[product_input]
    
#     similarity_score = list(enumerate(cosine_sim_matrix[product_index]))
#     similarity_score = sorted(similarity_score, key=lambda x: x[1],reverse=True)
#     similarity_score = similarity_score[1:10]
#     product_indices = [i[0] for i in similarity_score]
    
#     return (df['product_name'].iloc[product_indices])

In [ ]:
# recommend_products('Chocolate Sandwich Cookies')